In [3]:
import os
import csv
import pandas as pd

ruta_archivos = r"C:\Users\Usuario\Desktop\Programación\Proyecto Análisis de Datos Entrenos\Datos de entrenamiento"

def unificar_archivos(ruta_archivos):
    lista_datos = []
    
    for raiz, _, archivos in os.walk(ruta_archivos):
        for archivo in archivos:
            if archivo.endswith('.csv'):
                ruta_archivo = os.path.join(raiz, archivo)
                with open(ruta_archivo, newline='') as archivo_csv:
                    lector_csv = csv.reader(archivo_csv)
                    for fila in lector_csv:
                        lista_datos.append(fila)
    return lista_datos

lista_datos = unificar_archivos(ruta_archivos)

In [4]:
#Segundo paso: convertir la lista de listas en un dataframe de pandas
def convertir_a_pandas(lista_datos):
    df = pd.DataFrame(lista_datos)
    return df

df = convertir_a_pandas(lista_datos)

In [5]:
#Paso 3: eliminar las filas que no nos interesan
df.drop(df.loc[df[0] == "Atleta"].index, inplace=True)
df.drop(df.loc[df[0] == "Marcado con *: Solo encoder con Bluetooth"].index, inplace=True)
df.drop(df.loc[df[0] == "Email"].index, inplace=True)

In [6]:
#Paso 4: Determinar que nombre quiero para mis columnas y elimnar esas filas
df.columns = df.iloc[0]
df.drop(df.loc[df["Ejercicio"] == "Ejercicio"].index, inplace=True)

In [7]:
#Paso 5: Eliminar las columnas que no me interesan
df.drop("Comentarios", axis=1, inplace=True)

In [8]:
#Reseteo el index
df.reset_index(drop=True, inplace=True)

In [9]:
#Paso 6: Igualar todas las filas en las que no haya datos en la columna ejercicio, fecha, etc.

def rellenar_datos_faltantes(columna):
    valor_del_momento = df.loc[0, columna]
    for indice, valor in df[columna].iteritems():
        if valor == "":
            df.loc[indice, columna] = valor_del_momento
        else:
            valor_del_momento = valor

for columna in df.columns:
    rellenar_datos_faltantes(columna)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1344\2836985443.py:5: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for indice, valor in df[columna].iteritems():


In [10]:
#Cambio la fecha a formato date, elimino la hora
df["Fecha"] = pd.to_datetime(df["Fecha"], format="%d-%m-%Y %H:%M")
df["Fecha"] = df["Fecha"].dt.date

In [11]:
df.rename(columns={"nÂº Serie":"nº Serie", "nÂº Rep." : "nº Rep", "Velocidad mÃ¡xima (m/s)":"Velocidad máxima (m/s)"}, inplace=True)

In [12]:
#Elimino las filas que no tienen datos en la columna serie y en la columna repeticion
df = df.dropna(subset=["nº Serie"])
df = df.dropna(subset=["nº Rep"])

In [13]:
#Reseteo el index
df.reset_index(drop=True, inplace=True)

In [14]:
#Transformo los datos de las columnas que van a contener datos numericos
columnas_no_numericas = ["Ejercicio", "Fecha", "Tipo*"]
for columna in df.columns:
    if columna not in columnas_no_numericas:
        df[columna] = df[columna].str.replace(",", ".")
        df[columna] = pd.to_numeric(df[columna])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "pandas\_libs\lib.pyx", line 2369, in pandas._libs.lib.maybe_convert_numeric
ValueError: Unable to parse string "16.997.20"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Usuario\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Usuario\AppData\Local\Temp\ipykernel_1344\3402608575.py", line 6, in <module>
    df[columna] = pd.to_numeric(df[columna])
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\tools\numeric.py", line 185, in to_numeric
    values, _ = lib.maybe_convert_numeric(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\lib.pyx", line 2411, in pandas._libs.lib.maybe_convert_numeric
ValueError: Unable to parse string "16.997.20" at po

In [15]:
#Convierto columna fecha a formato date
df["Fecha"] = pd.to_datetime(df["Fecha"], format="%Y-%m-%d")

In [16]:
#Añado nuevas columnas con datos que me interesan
df_fatiga = df.groupby(["Ejercicio", "Fecha", "nº Serie", "Peso (kg)"]).agg({"VMP (m/s)" : ["min", "max"]})
df_fatiga["Fatiga"] = df_fatiga["VMP (m/s)"]["max"] - df_fatiga["VMP (m/s)"]["min"]
df_fatiga["Porcentaje de fatiga"] = df_fatiga["Fatiga"] / df_fatiga["VMP (m/s)"]["max"] * 100
df_fatiga.reset_index(inplace=True)

3             Ejercicio      Fecha nº Serie Peso (kg) VMP (m/s)       Fatiga  \
                                                            min   max          
0         CMJ FREE ARMS 2022-12-02        1      86.0      1.69  1.69   0.00   
1         CMJ FREE ARMS 2022-12-02        2      86.0      1.81  1.81   0.00   
2         CMJ FREE ARMS 2022-12-02        3      86.0      1.73  1.73   0.00   
3         CMJ FREE ARMS 2022-12-02        4      86.0      0.53  1.70   1.17   
4         CMJ FREE ARMS 2023-02-13        1      84.0      1.72  1.72   0.00   
..                  ...        ...      ...       ...       ...   ...    ...   
204  SENTADILLA FRONTAL 2023-03-06        4      85.0      0.45  0.59   0.14   
205  SENTADILLA FRONTAL 2023-03-08        1      75.0      0.56  0.60   0.04   
206  SENTADILLA FRONTAL 2023-03-08        2      75.0      0.54  0.67   0.13   
207  SENTADILLA FRONTAL 2023-03-14        1      90.0      0.33  0.33   0.00   
208  SENTADILLA FRONTAL 2023-03-14        2      90.0      0.40  0.40   0.00   

3   Porcentaje de fatiga  
                          
0               0.000000  
1               0.000000  
2               0.000000  
3              68.823529  
4               0.000000  
..                   ...  
204            23.728814  
205             6.666667  
206            19.402985  
207             0.000000  
208             0.000000  

[209 rows x 8 columns]

In [17]:
#Creo la función para eliminar las filas que sean menores a las filas con máximo peso
def eliminar_filas(df):
    max_peso = df['Peso (kg)'].max()
    return df[df['Peso (kg)'] == max_peso]

df_fatiga = df_fatiga.groupby(["Ejercicio", "Fecha"]).apply(eliminar_filas)
df_fatiga.reset_index(drop=True, inplace=True)
df_fatiga

3             Ejercicio      Fecha nº Serie Peso (kg) VMP (m/s)       Fatiga  \
                                                            min   max          
0         CMJ FREE ARMS 2022-12-02        1      86.0      1.69  1.69   0.00   
1         CMJ FREE ARMS 2022-12-02        2      86.0      1.81  1.81   0.00   
2         CMJ FREE ARMS 2022-12-02        3      86.0      1.73  1.73   0.00   
3         CMJ FREE ARMS 2022-12-02        4      86.0      0.53  1.70   1.17   
4         CMJ FREE ARMS 2023-02-13        1      84.0      1.72  1.72   0.00   
..                  ...        ...      ...       ...       ...   ...    ...   
126  SENTADILLA FRONTAL 2023-03-06        4      85.0      0.45  0.59   0.14   
127  SENTADILLA FRONTAL 2023-03-08        1      75.0      0.56  0.60   0.04   
128  SENTADILLA FRONTAL 2023-03-08        2      75.0      0.54  0.67   0.13   
129  SENTADILLA FRONTAL 2023-03-14        1      90.0      0.33  0.33   0.00   
130  SENTADILLA FRONTAL 2023-03-14        2      90.0      0.40  0.40   0.00   

3   Porcentaje de fatiga  
                          
0               0.000000  
1               0.000000  
2               0.000000  
3              68.823529  
4               0.000000  
..                   ...  
126            23.728814  
127             6.666667  
128            19.402985  
129             0.000000  
130             0.000000  

[131 rows x 8 columns]

In [18]:
df_maxVMP_maxpeso = df_fatiga.groupby(["Ejercicio", "Fecha", "Peso (kg)"]).agg({("VMP (m/s)", "max") : ["max"]})
df_maxVMP_maxpeso.reset_index(inplace=True)
df_maxVMP_maxpeso.columns = ["Ejercicio", "Fecha", "Peso (kg)", "VMP (m/s)"]
df_maxVMP_maxpeso

,Ejercicio,Fecha,Peso (kg),VMP (m/s)
0,CMJ FREE ARMS,2022-12-02,86.0,1.81
1,CMJ FREE ARMS,2023-02-13,84.0,1.72
2,CMJ FREE ARMS,2023-02-14,84.0,1.96
3,CMJ FREE ARMS,2023-02-15,84.0,2.03
4,CMJ FREE ARMS,2023-02-19,84.0,2.05
5,CMJ FREE ARMS,2023-02-20,84.0,2.23
6,CMJ FREE ARMS,2023-02-21,84.0,2.03
7,CMJ FREE ARMS,2023-02-23,84.0,1.81
8,CMJ FREE ARMS,2023-02-27,884.0,1.96
9,CMJ FREE ARMS,2023-03-02,84.0,2.13


In [19]:
#Obtengo la media de los porcentajes de fatiga por fecha y ME FALTA HACERLO POR EJERCICIO
df_fatiga.columns = ["Ejercicio", "Fecha", "nº Serie", "Peso (kg)", "VMP (m/s) min", "VMP (m/s) max", "Fatiga", "Porcentaje de fatiga"]
df_fatiga_media = df_fatiga.groupby(["Fecha"]).agg({"Porcentaje de fatiga" : "mean"})
df_fatiga_media.reset_index(inplace=True)
df_fatiga_media

,Fecha,Porcentaje de fatiga
0,2022-12-02,17.205882
1,2022-12-14,5.883546
2,2022-12-18,12.935006
3,2022-12-22,9.151377
4,2022-12-26,9.834229
5,2022-12-28,0.000000
6,2023-01-03,5.617284
7,2023-01-05,10.197368
8,2023-01-10,11.958263
9,2023-01-15,0.000000


In [21]:
#Convierto dataframe a excel
nombre_archivo = "Datos.xlsx"
writer = pd.ExcelWriter(nombre_archivo, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
df_fatiga_media.to_excel(writer, sheet_name='Sheet2')
df_maxVMP_maxpeso.to_excel(writer, sheet_name='Sheet3')
writer.save()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1344\1887950985.py:7: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
